<a href="https://colab.research.google.com/github/toshiikuoo/puclic/blob/master/%E6%A0%AA%E4%BE%A1%E7%9B%B8%E9%96%A2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#必要なライブラリimport
!pip install lxml html5lib beautifulsoup4

import pandas as pd
from pandas import Series,DataFrame
from pandas_datareader import DataReader

import numpy as np

from datetime import datetime

from scipy.stats.stats import pearsonr
import itertools

# Install yfinance package.
!pip install yfinance
 
# Import yfinance
import yfinance as yf  

  Created wheel for yfinance: filename=yfinance-0.1.50-py2.py3-none-any.whl size=21556 sha256=48984b9d2c25a459c790a6f67a85999c7b42a05f3d86cef1b737be63e4c50b31
  Stored in directory: /root/.cache/pip/wheels/19/2a/e6/b4d4cc977576e18220a36866acf2f2cf7241f16af5f8795b17
Successfully built yfinance


In [2]:
#S&P全銘柄のリストを作成
url="https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
sp500_list=pd.read_html(url)[0].Symbol.values.tolist()
len(sp500_list)

505

In [3]:
#yahooでは銘柄シンボルの"."は"-"に変えないとエラーになるので置換
#"."を含む要素を確認
sp500_list_in=[i for i in sp500_list if "." in i]
sp500_list_in

['BRK.B', 'BF.B']

In [0]:
#"."を"-"に置換
sp500_list_yahoo=[i.replace(".","-")for i in sp500_list]

In [5]:
#確認のため"-"の含まれる要素を表示"
sp500_list_yahoo_in=[i for i in sp500_list_yahoo if "-" in i]
print(sp500_list_yahoo_in)

['BRK-B', 'BF-B']


In [6]:
len(sp500_list_yahoo)

505

In [7]:
#sp500銘柄の終値を１つのDataFrameに格納
try:
  close_sp500_list=yf.download(sp500_list_yahoo,'2019-10-04','2019-11-01')["Adj Close"]
except KeyError:
  print('No such key')

[*********************100%***********************]  505 of 505 completed


In [8]:
len(list(itertools.combinations(sp500_list,2)))


127260

In [0]:
#列ごとのペアで相関を算出


#算出した相関を入れるための辞書型作成
correlations={}


#相関を算出

for cola,colb in itertools.combinations(sp500_list_yahoo,2):
  nas=np.logical_or(np.isnan(close_sp500_list.loc[:,cola]),np.isnan(close_sp500_list.loc[:,colb]))
  try:
    correlations[cola + '__'+ colb]=pearsonr(close_sp500_list.loc[:,cola][~nas],close_sp500_list.loc[:,colb][~nas])
  except ValueError:
    pass
    

In [10]:
#出力された結果"correlations"はリスト形式なのでDataFrameに変換
result=DataFrame.from_dict(correlations,orient='index')
result.columns=['PCC','p-value']
print(result.sort_values('PCC'))

                  PCC       p-value
BIIB__TRV   -0.975397  2.869659e-13
BIIB__HAS   -0.975153  3.133046e-13
ALLE__TWTR  -0.974004  4.686860e-13
INTC__VTR   -0.972172  8.591312e-13
LRCX__TWTR  -0.971212  1.161760e-12
...               ...           ...
BMY__CELG    0.995908  3.010545e-20
HCA__LEG     0.996207  1.523031e-20
CNC__WCG     0.998694  1.041787e-24
BBT__STI     0.998977  1.159837e-25
GOOGL__GOOG  0.999658  6.000199e-30

[126253 rows x 2 columns]


以下、つまづいたところ

* [データが二つ未満の列があり、pearsonrが機能しない](https://colab.research.google.com/drive/1BdfWKBROd_oAsrIvP2FLNEG3tTkbWhli#scrollTo=3iFGD5gSH5EH&line=4&uniqifier=1)  
→try exceptで無視

エラーコード
```
#列ごとのペアで相関を算出



#算出した相関を入れるための辞書型作成
correlations={}

#相関を算出
for cola,colb in itertools.combinations(sp500_list_yahoo,2):
  nas=np.logical_or(np.isnan(close_sp500_list.loc[:,cola]),np.isnan(close_sp500_list.loc[:,colb]))
 # try:
  correlations[cola + '__'+ colb]=pearsonr(close_sp500_list.loc[:,cola][~nas],close_sp500_list.loc[:,colb][~nas])
#  except ValueError:
#    print('No such key')

---------------------------------------------------------------------------
ValueError                                Traceback (most recent call last)
<ipython-input-18-0eb82233969b> in <module>()
      5   nas=np.logical_or(np.isnan(close_sp500_list.loc[:,cola]),np.isnan(close_sp500_list.loc[:,colb]))
      6  # try:
----> 7   correlations[cola + '__'+ colb]=pearsonr(close_sp500_list.loc[:,cola][~nas],close_sp500_list.loc[:,colb][~nas])
      8 #  except ValueError:
      9 #    print('No such key')

/usr/local/lib/python3.6/dist-packages/scipy/stats/stats.py in pearsonr(x, y)
   3390 
   3391     if n < 2:
-> 3392         raise ValueError('x and y must have length at least 2.')
   3393 
   3394     x = np.asarray(x)

ValueError: x and y must have length at least 2.
```
エラーの列の特定
```
#NaNが２つ以上の行とその数を表示
for i in close_sp500_list:
  if close_sp500_list.loc[:,i].isnull().sum()>1:
    print(i)
    print(close_sp500_list.loc[:,i].isnull().sum())
```